# Análisis de riesgo crediticio
**Objetivo:** Limpiar los datos del CSV original.
**Autor:** Eberth Gianfranco Rojas Barbaran
## 1. Carga de librerías

In [2]:
import pandas as pd
import numpy as np

## 2. Extracción y análisis del archivo CSV

In [3]:
# 1. Cargamos la data
archivo_csv="../data/raw/german_credit_data.csv"
df=pd.read_csv(archivo_csv)

In [4]:
# 2 Tamaño del DataFrame
print("--- TAMAÑO (Filas, Columnas) ---")
print(df.shape)

--- TAMAÑO (Filas, Columnas) ---
(1000, 10)


In [5]:
# 3. Informacion del DataFrame
print("\n--- INFORMACIÓN TÉCNICA ---")
print(df.info())


--- INFORMACIÓN TÉCNICA ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   int64 
 1   Age               1000 non-null   int64 
 2   Sex               1000 non-null   object
 3   Job               1000 non-null   int64 
 4   Housing           1000 non-null   object
 5   Saving accounts   817 non-null    object
 6   Checking account  606 non-null    object
 7   Credit amount     1000 non-null   int64 
 8   Duration          1000 non-null   int64 
 9   Purpose           1000 non-null   object
dtypes: int64(5), object(5)
memory usage: 78.3+ KB
None


## 3. Limpieza de los datos

In [6]:
# 1. VER EL PROBLEMA EXACTO DE NULOS
print("---Nulos ANTES de arreglar---")
print(df.isnull().sum())

---Nulos ANTES de arreglar---
Unnamed: 0            0
Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
dtype: int64


In [7]:
# 2. ELIMINAR COLUMNA INNECESARIA
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [8]:
# 3. LA SOLUCIÓN (Imputación)
df['Saving accounts'] = df['Saving accounts'].fillna('Sin Cuenta')
df['Checking account'] = df['Checking account'].fillna('Sin Cuenta')

In [9]:
# 4. VERIFICAR QUE QUEDÓ LIMPIO
print("\n--- Nulos DESPUÉS de arreglar ---")
print(df.isnull().sum())


--- Nulos DESPUÉS de arreglar ---
Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
dtype: int64


## 4. Traducción de los datos

In [10]:
# --- PASO 1: RENOMBRAR COLUMNAS ---
mapa_nombres = {
    'Age': 'Edad',
    'Sex': 'Genero',
    'Job': 'Id_Trabajo',
    'Housing': 'Tipo_Vivienda',
    'Saving accounts': 'Cta_Ahorro',
    'Checking account': 'Cta_Corriente',
    'Credit amount': 'Monto_Credito',
    'Duration': 'Plazo_Meses',
    'Purpose': 'Propósito',
}

# Ejecutamos el cambio
df = df.rename(columns=mapa_nombres)

# Muestra las nuevas cabeceras para confirmar
print(df.columns)

Index(['Edad', 'Genero', 'Id_Trabajo', 'Tipo_Vivienda', 'Cta_Ahorro',
       'Cta_Corriente', 'Monto_Credito', 'Plazo_Meses', 'Propósito'],
      dtype='object')


In [ ]:
# --- PASO 2: TRADUCIR VALORES INTERNOS ---

# A. Traducir Vivienda

diccionario_vivienda = {
    'own': 'Propia',
    'rent': 'Alquiler',
    'free': 'Cedida/Gratis'
}

df['Tipo_Vivienda'] = df['Tipo_Vivienda'].map(diccionario_vivienda)

print(df['Tipo_Vivienda'].value_counts())

# B. Traducir Género

diccionario_genero = {
    'male': 'Masculino',
    'female': 'Femenino'
}

df['Genero'] = df['Genero'].map(diccionario_genero)

print(df['Genero'].value_counts())

# C. Traducir Finalidad

def limpiar_finalidad(texto):
    # Convertimos a texto y a minúsculas para evitar errores 
   
    # Buscamos palabras clave dentro del texto
    if 'car' in texto:
        return 'Vehículo'
    elif 'radio' in texto or 'tv' in texto:
        return 'Electrodomésticos'
    elif 'furniture' in texto or 'equipment' in texto:
        return 'Muebles/Equip.'
    elif 'education' in texto:
        return 'Educación'
    elif 'business' in texto:
        return 'Negocios'
    elif 'repairs' in texto:
        return 'Reparaciones'
    else:
        # Si no coincide con nada anterior, lo metemos en 'Otros'
        return 'Otros/Varios'

df['Propósito'] = df['Propósito'].apply(limpiar_finalidad)

print(df['Propósito'].value_counts())

# D. Traducir Cuentas (Sirve para Ahorro y Corriente)

mapa_dinero = {
    'little': 'Poco',
    'moderate': 'Moderado',
    'quite rich': 'Bastante rico',
    'rich': 'Rico',
    'Sin Cuenta': 'Sin Cuenta' 
}

# Aplicamos el mismo mapa a las dos columnas
df['Cta_Ahorro'] = df['Cta_Ahorro'].map(mapa_dinero)
df['Cta_Corriente'] = df['Cta_Corriente'].map(mapa_dinero)

print(df['Cta_Ahorro'].value_counts())
print(df['Cta_Corriente'].value_counts())

# E. Traducir Trabajo (De números a palabras reales)

mapa_trabajo = {
    0: 'Desempleado/eventual',
    1: 'Operativo/Técnico',
    2: 'Empleado Dependiente',
    3: 'Independiente/Gerencia'
}

df['Id_Trabajo'] = df['Id_Trabajo'].map(mapa_trabajo)
print(df['Id_Trabajo'].value_counts())

# F. Revisión final

print("--- ESTADO FINAL DE LA TABLA ---")
df.info()

Tipo_Vivienda
Propia           713
Alquiler         179
Cedida/Gratis    108
Name: count, dtype: int64
Genero
Masculino    690
Femenino     310
Name: count, dtype: int64
Propósito
Vehículo             337
Electrodomésticos    280
Muebles/Equip.       181
Negocios              97
Educación             59
Otros/Varios          24
Reparaciones          22
Name: count, dtype: int64
Cta_Ahorro
Poco             603
Sin Cuenta       183
Moderado         103
Bastante rico     63
Rico              48
Name: count, dtype: int64
Cta_Corriente
Sin Cuenta    394
Poco          274
Moderado      269
Rico           63
Name: count, dtype: int64
Id_Trabajo
Empleado Dependiente      630
Operativo/Técnico         200
Independiente/Gerencia    148
Desempleado/eventual       22
Name: count, dtype: int64
--- ESTADO FINAL DE LA TABLA ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         -------

## 5. Creación del motor de riesgo

In [ ]:
# --- PASO 5: CÁLCULO DE RIESGO (MODULARIZADO) ---
import sys
import os

# 1. Ubicar los 'scripts'

sys.path.append(os.path.abspath('..'))

# 2. Importamos las funciones desde el archivo que acabas de crear

from scripts.reglas_negocio import obtener_score, definir_etiqueta

print("✅ Funciones de negocio importadas correctamente desde scripts/reglas_negocio.py")

# axis=1 es vital para ir fila por fila
df['Score'] = df.apply(obtener_score, axis=1)
 
# Aplicamos la etiqueta sobre la columna de números
df['Resultado_Riesgo'] = df['Score'].apply(definir_etiqueta)

# --- VER RESULTADOS ---
print("--- Distribución del Riesgo (Corte < 82) ---")
print(df['Resultado_Riesgo'].value_counts())

print("\n--- Muestra de Datos ---")
cols_interes = ['Edad', 'Propósito', 'Score', 'Resultado_Riesgo']
display(df[cols_interes].head(5))

--- Distribución del Riesgo (Corte < 82) ---
Resultado_Riesgo
Normal        615
Deficiente    385
Name: count, dtype: int64

--- ¡Mira! Ahora tienes Score Y Resultado ---


,Edad,Propósito,Score,Resultado_Riesgo
950,40,Negocios,70,Deficiente
399,43,Electrodomésticos,80,Deficiente
169,31,Negocios,100,Normal
742,41,Electrodomésticos,90,Normal
662,50,Muebles/Equip.,100,Normal
871,46,Vehículo,100,Normal
355,23,Vehículo,80,Deficiente
586,36,Vehículo,80,Deficiente
911,25,Muebles/Equip.,90,Normal
318,32,Educación,100,Normal


## 6. Exportar el csv 

In [13]:
import os
ruta_salida = '../data/processed/german_credit_riesgo_final.csv'
df.to_csv(ruta_salida, index=False)
print(f" Archivo guardado en: {ruta_salida}")

 Archivo guardado en: ../data/processed/german_credit_riesgo_final.csv
